In [1]:
import numpy as np 
import pandas as pd
import scipy as sc
from auxfunc import *
from sentropies import *

IndentationError: unindent does not match any outer indentation level (sentropies.py, line 124)

In [ ]:
def sentropies_df(df, type = "total" , base = 2):

    dims = df.shape

    if(not isinstance(df,pd.DataFrame)):
        sys.exit("Can only work with Data Frames!")

    if(dims[1] == 0 or dims[0] == 0): 
        sys.exit("Can only work with non-empty DataFrames!")

    if (not(all(df.dtypes)=='category')):
        warnings.warn("Discretizing data from X DataFrame before entropy calculation!") #' Throwing a Warning for communicating a discretization of data
        df = discretization(df)

    H_Uxi = df.apply((lambda x : np.log2(len(pd.unique(x)))), axis = 0).values
    H_Pxi = df.apply(ent,axis=0).values

    if (dims[1]==1):

        warnings.warn("Single variable: providing only entropy")
        VI_Pxi = H_Pxi

    else:

        VI_Pxi = condentropies(df)

    if (type == "total") : # TOTAL decomposition & Aggregates

        if (any(df.columns.isnull())):
            #' Delete names and append new ones
            warnings.warn("No names for columns: providing variable names!")
            #Apply dummy names 
            df.columns = list(map(lambda x: "x"+str(x), range(len(df.columns))))

                #'DataFrame Creation
        edf = pd.DataFrame({'Name': list(df.columns) , 'H_Uxi': H_Uxi , 'H_Pxi':H_Pxi ,'DeltaH_Pxi': H_Uxi - H_Pxi ,'M_Pxi': H_Pxi - VI_Pxi , 'VI_Pxi': VI_Pxi}, columns = ['Name','H_Uxi','H_Pxi','DeltaH_Pxi','M_Pxi','VI_Pxi'])
        edf = edf.set_index('Name')
        edf.loc['All'] = edf.sum(axis = 0)


    else :  #return only an aggregate with the DUAL total correlation D_Px

        H_Ux = np.sum(H_Uxi)
        H_Px = ent(sjoin(df,lis = df.columns))
        #H_Px = np.sum(H_Pxi)
        VI_Px = np.sum(VI_Pxi)

        edf = pd.DataFrame({'Name' : ['All'] , 'H_Ux' : H_Ux , 'H_Px'  : H_Px , 'DeltaH_Px' : (H_Ux - H_Px) ,'D_Px' : H_Px - VI_Px , 'VI_Px' : VI_Px }, columns = ['Name','H_Ux','H_Px','DeltaH_Px','D_Px','VI_Px'])
        edf = edf.set_index('Name')

    return edf


In [10]:
df = pd.read_excel('titanic.xlsx')

In [11]:
df1 = discretization(df)

/Users/jaime.de.los.rios/Documents/Jupyter Prueba Fran/auxfunc.py:28: UserWarning: Discretizing data!
  warnings.warn("Discretizing data!")


In [14]:
nb = 5
li = list() ; dims = list() 

df.apply((lambda x : dims.append(len(pd.unique(x)))))
for i in range(len(df.columns)):li.append(pd.Series(df[df.columns[i]]))

m = pd.crosstab(index = li[0:len(li)-1], columns = li[len(li)-1], dropna= False)

In [22]:
H_Uxi = df.apply((lambda x : np.log2(len(pd.unique(x)))), axis = 0).values

In [23]:
H_Pxi = df.apply(ent,axis=0).values

In [24]:
a = df['survived']

In [27]:
a.value_counts()
p_data = a.value_counts()/len(a)

In [30]:
H_Pxi

array([1.45824504, 0.41206192, 0.9244984 , 0.95746035])

In [31]:
H_Uxi

array([1.5849625, 1.       , 1.       , 1.       ])

In [32]:
VI_Pxi = condentropies(df)

In [35]:
cond = list() ; ncol = df.columns
total = ent(sjoin(df,lis = ncol),base = 2)

In [40]:
p = sjoin(df,lis = ncol,sep=',')

In [42]:
p.value_counts()/len(df)

3rd class,adults,man,no       0.294073
2nd class,adults,man,no       0.117021
1st class,adults,women,yes    0.106383
1st class,adults,man,no       0.089666
3rd class,adults,women,no     0.067629
2nd class,adults,women,yes    0.060790
3rd class,adults,women,yes    0.057751
3rd class,adults,man,yes      0.056991
1st class,adults,man,yes      0.043313
3rd class,child,man,no        0.026596
3rd class,child,women,no      0.012918
3rd class,child,women,yes     0.010638
2nd class,adults,man,yes      0.010638
2nd class,adults,women,no     0.009878
3rd class,child,man,yes       0.009878
2nd class,child,women,yes     0.009878
2nd class,child,man,yes       0.008359
1st class,child,man,yes       0.003799
1st class,adults,women,no     0.003040
1st class,child,women,yes     0.000760
Name: class, dtype: float64

In [45]:
edf = sentropies_df(df,type='dual')

/Users/jaime.de.los.rios/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Discretizing data from X DataFrame before entropy calculation!
  if sys.path[0] == '':
/Users/jaime.de.los.rios/Documents/Jupyter Prueba Fran/auxfunc.py:28: UserWarning: Discretizing data!
  warnings.warn("Discretizing data!")


In [46]:
edf

,H_Ux,H_Px,DeltaH_Px,D_Px,VI_Px
Name,,,,,
All,4.584963,3.393681,1.191281,0.410831,2.982851


In [65]:
dfb = pd.read_csv('Breast_data.csv',delimiter=',',na_values=0)
dfb.drop(labels='Class',axis = 1)

In [87]:
edf = sentropies_df(ert,'dual')

/Users/jaime.de.los.rios/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Discretizing data from X DataFrame before entropy calculation!
  if sys.path[0] == '':
/Users/jaime.de.los.rios/Documents/Jupyter Prueba Fran/auxfunc.py:28: UserWarning: Discretizing data!
  warnings.warn("Discretizing data!")


In [89]:
edf

,H_Ux,H_Px,DeltaH_Px,D_Px,VI_Px
Name,,,,,
All,19.191599,8.061969,11.12963,6.824586,1.237384


In [90]:
derivedSplitSmetCoords = ["DeltaH_Pxi", "M_Pxi", "VI_Pxi", "Name"] #multivariate entropic coordinates
# Note that, strictly speaking, VI_Pxi is not derived

#' Caracterization of the variables in a DataFrame Variables of aggregated entropies

aggregateSmetCoords= ["H_Ux", "DeltaH_Px", "M_Px", "VI_Px"] #source multivariate aggregate coordinates
dualAggregateSmetCoords= ["H_Ux", "DeltaH_Px", "D_Px", "VI_Px"] # SMET coords without C_P_X

#' Caracterization of the variables in a DataFrame of Channel Multivariate Entropies
cmetEntropicCoords = ["HU", "HP", "DeltaH_P", "M_P", "VI_P"]



#' Functions to detect SMET coordinates

def hasSplitSmetCoords(df):
	'''
	A function to detect if the source multivariate split entropies are present: this enables working out the multivariate split entropic coordinates
	Function -> sentropies_df(type = total)
	splitSmetCoords  = ["Name", "H_Uxi", "H_Pxi", "VI_Pxi"] # multivariate split entropies
	
	@param -> DataFrame with entropic values
	@return -> boolean (true or false)
	'''
	return all(df.columns.isin(derivedSplitSmetCoords))




def hasAggregateSmetCoords(df):	
	
	'''
	SMET
	A function to detect if the source multivariate aggregate entropic coordinates are present
	Function -> sentropies_df(type = 'total') SUM
	aggregateSmetCoords= ["H_Ux", "DeltaH_Px", "M_Px", "VI_Px"] #source multivariate aggregate coordinates

	@param -> DataFrame with entropic values
	@return -> boolean (true or false)

	'''
	return all(df.columns.isin(aggregateSmetCoords))


def hasDualAggregateSmetCoords(df):
	
	'''
	SMET
	A function to detect if the source multivariate aggregate entropic coordinates are present
	Function -> sentropies_df(type = 'dual')
	dualAggregateSmetCoords= ["H_Ux", "DeltaH_Px", "D_Px", "VI_Px"] # SMET coords without C_P_X

	@param -> DataFrame with entropic values
	@return -> boolean (true or false)

	'''
	return all(df.columns.isin(dualAggregateSmetCoords))


#' Functions to detect CMET coordinates

def hasCmetEntropicCoords(df):
	
	'''
	CMET
	A function to detect if the the channel multivariate entropic coordinates are present
	Function -> jentropies_df
	cmetEntropicCoords = ["H_U", "H_P", "VI_P", "DeltaH_P", "M_P"]

	@param -> DataFrame with entropic values
	@return -> boolean (true or false)

	'''
	return all(df.columns.isin(cmetEntropicCoords))

In [91]:
hasAggregateSmetCoords(edf)

False

In [92]:
edf

,H_Ux,H_Px,DeltaH_Px,D_Px,VI_Px
Name,,,,,
All,19.191599,8.061969,11.12963,6.824586,1.237384


In [93]:
aggregateSmetCoords

['H_Ux', 'DeltaH_Px', 'M_Px', 'VI_Px']

In [97]:
a = edf.columns.isin(dualAggregateSmetCoords)

In [101]:
if(edf.columns.isin(dualAggregateSmetCoords).sum() == len(dualAggregateSmetCoords)):
    print('true')
else: 
    print('false')

true
